In [1]:
# Set project root
import os
import sys

# Manually set the path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import least_squares

from src.geometry import derive_metric_homography
from src.matching import template_match
from src.data import load_measurements_from_yaml
from src.utils import load_calibration_json, load_rgb

In [3]:
# Load measurement data
data = load_measurements_from_yaml("../assets/measurements.yaml")

# Load camera calibration
K, dist_coeffs, image_size = load_calibration_json("../assets/camera_calibration.json")

In [4]:
def compute_homographies(scene_id, scenes):
    """
    This function analyzes a scene by executing the following steps:

    1. Load the scene and the corresponding templates.
    2. For each template, perform template matching against the scene image.
    3. Compute the homography.
    """
    # Load scene and templates
    scene = data.get_scene(scene_id)
    templates = [data.get_template(t_id) for t_id in scenes[scene_id]]

    # Iterate over templates
    results = {}
    for template in templates:
        scene_path = os.path.join(project_root, scene.path)
        template_path = os.path.join(project_root, template.path)

        # Compute pixel-pixel homography
        H, mask, t_shape, reprojection_error = template_match(template_path, scene_path, extract_method='SIFT', match_method='BF', plot=False)

        template_size_px = plt.imread(template_path).shape[:2]

        # Derive the metric-pixel homography
        H_metric = derive_metric_homography(
            H_px=H,
            template_size_px=template_size_px,
            template_size_metric=(template.height, template.width)
        )

        results[template.id] = {
            'H_px': H.tolist(),
            'H_metric': H_metric.tolist()
        }

    return results

In [5]:
class Calibration:
    """
    Adapted Zhang's camera calibration method for estimating intrinsic parameters
    from homographies between multiple planar templates in a single scene.
    """
    
    def __init__(self):
        self.intrinsic_matrix = None
        self.homographies = []
    
    def _build_v_ij(self, H: np.ndarray, i: int, j: int) -> np.ndarray:
        """
        Compute the vector v_ij used in the Zhang-style calibration constraint:
        v_ij = [h1i*h1j, h1i*h2j + h2i*h1j, h2i*h2j, h3i*h1j + h1i*h3j, h3i*h2j + h2i*h3j, h3i*h3j]
        
        Args:
            H: homography matrix (3x3)
            i, j: column indices for the constraint
            
        Returns:
            v_ij: constraint vector (6,)
        """
        h = H.T  # transpose to access columns easily
        return np.array([
            h[i, 0] * h[j, 0],
            h[i, 0] * h[j, 1] + h[i, 1] * h[j, 0],
            h[i, 1] * h[j, 1],
            h[i, 0] * h[j, 2] + h[i, 2] * h[j, 0],
            h[i, 1] * h[j, 2] + h[i, 2] * h[j, 1],
            h[i, 2] * h[j, 2]
        ])
    
    def add_homography(self, H: np.ndarray):
        """
        Add a homography to the calibration dataset.
        
        Args:
            H: homography matrix (3x3)
        """
        if H.shape != (3, 3):
            raise ValueError("Homography must be a 3x3 matrix")
        self.homographies.append(H.copy())
    
    def add_homographies(self, Hs: list[np.ndarray]):
        """
        Add multiple homographies to the calibration dataset.
        
        Args:
            Hs: list of homography matrices (each 3x3)
        """
        for H in Hs:
            self.add_homography(H)
    
    def calibrate(self) -> np.ndarray:
        """
        Estimate the intrinsic matrix K from the stored homographies.
        
        Returns:
            K: intrinsic matrix (3x3)
            
        Raises:
            ValueError: if fewer than 3 homographies are available
        """
        if len(self.homographies) < 3:
            raise ValueError("At least 3 homographies are required for calibration")
        
        V = []
        for H in self.homographies:
            H_norm = H / H[2, 2]  # Normalize
            v12 = self._build_v_ij(H_norm, 0, 1)  # v_12
            v11 = self._build_v_ij(H_norm, 0, 0)  # v_11
            v22 = self._build_v_ij(H_norm, 1, 1)  # v_22
            V.append(v12)
            V.append(v11 - v22)

        V = np.vstack(V)
        
        # Solve Vb = 0 using SVD
        _, _, Vh = np.linalg.svd(V)
        b = Vh[-1, :]  # last row corresponds to smallest singular value

        # b = [B11, B12, B22, B13, B23, B33]
        B11, B12, B22, B13, B23, B33 = b

        # Compute intrinsic parameters from B
        # cy = (B12 * B13 - B11 * B23) / (B11 * B22 - B12**2)
        # scale = B33 - (B13**2 + cy * (B12 * B13 - B11 * B23)) / B11
        # fx = np.sqrt(scale / B11)
        # fy = np.sqrt(scale * B11 / (B11 * B22 - B12**2))
        # s = -B12 * fx**2 * fy / scale
        # cx = s * cy / fy - B13 * fx**2 / scale

        # Alternative (?)
        w1, w2, w3, w4, w5, w6 = b
        
        w = w1 * w3 * w6 - w2 ** 2 * w6 - w1 * w5 ** 2 + 2 * w2 * w4 * w5 - w3 * w4 ** 2
        d = w1 * w3 - w2 ** 2

        fx = np.sqrt(w / (d * w1))
        fy = np.sqrt(w / (d ** 2) * w1)
        cx = (w2 * w5 - w3 * w4) / d
        cy = (w2 * w4 - w1 * w5) / d
        s = np.sqrt(w / (d ** 2 * w1)) * w

        # Construct the intrinsic matrix
        K = np.array([
            [fx, s, cx],
            [0, fy, cy],
            [0, 0, 1]
        ])
        
        self.intrinsic_matrix = K
        return K
    
    def get_intrinsic_matrix(self) -> np.ndarray:
        """
        Get the estimated intrinsic matrix.
        
        Returns:
            K: intrinsic matrix (3x3) or None if calibration hasn't been performed
        """
        return self.intrinsic_matrix
    
    def reset(self):
        """
        Clear all stored homographies and reset the calibration state.
        """
        self.homographies.clear()
        self.intrinsic_matrix = None
    
    def get_num_homographies(self) -> int:
        """
        Get the number of stored homographies.
        
        Returns:
            Number of homographies
        """
        return len(self.homographies)

In [6]:
from scipy.spatial.transform import Rotation as R

def simulate_homography(K: np.ndarray, R_wc: np.ndarray, t_wc: np.ndarray) -> np.ndarray:
    """
    Simulate homography H = K [r1 r2 t] for a plane Z=0 in world coordinates
    """
    r1 = R_wc[:, 0]
    r2 = R_wc[:, 1]
    t = t_wc
    H = K @ np.column_stack([r1, r2, t])
    return H / H[2, 2]  # normalize

def generate_synthetic_data(n: int = 5, seed: int = 42):
    np.random.seed(seed)

    # Ground truth intrinsic parameters
    fx, fy = 1200, 1100
    cx, cy = 640, 360
    skew = 0
    K_true = np.array([
        [fx, skew, cx],
        [0,  fy,   cy],
        [0,   0,    1]
    ])

    homographies = []

    for _ in range(n):
        # Random rotation (small perturbations)
        rot_vec = np.random.randn(3) * 0.1  # small angle
        R_wc = R.from_rotvec(rot_vec).as_matrix()

        # Random translation (positive Z to stay in front of camera)
        t_wc = np.array([
            np.random.uniform(-0.1, 0.1), 
            np.random.uniform(-0.1, 0.1), 
            np.random.uniform(1.5, 2.5)
        ])

        # Generate homography
        H = simulate_homography(K_true, R_wc, t_wc)
        homographies.append(H)

    return K_true, homographies


# --- Run the simulation and estimation ---

K_gt, homographies = generate_synthetic_data(n=3)

calib = Calibration()
calib.add_homographies(homographies)
K_est = calib.calibrate()

# Print results
np.set_printoptions(precision=3, suppress=True)
print("Ground Truth K:\n", K_gt)
print("\nEstimated K:\n", K_est)
print("\nAbsolute Error:")
print(np.abs(K_est - K_gt))

Ground Truth K:
 [[1200    0  640]
 [   0 1100  360]
 [   0    0    1]]

Estimated K:
 [[1200.    0.  640.]
 [   0. 1100.  360.]
 [   0.    0.    1.]]

Absolute Error:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [7]:
# Analyze scene
results = compute_homographies("S12", data.get_all_scenes())

# Use the computed homographies to calibrate the camera
homographies_metric = []
for template_id, result in results.items():
    H_metric = np.array(result['H_metric'])
    homographies_metric.append(H_metric)

calib.reset()
calib.add_homographies(homographies_metric)
K_est_from_homographies_metric = calib.calibrate()

# Print estimated intrinsic matrix
print("\nEstimated K from metric homographies:\n", K_est_from_homographies_metric)

# Print the correct intrinsic matrix for comparison
print("\nCorrect K from calibration file:\n", K)


Estimated K from metric homographies:
 [[694.442  -0.    429.994]
 [  0.    678.093 629.134]
 [  0.      0.      1.   ]]

Correct K from calibration file:
 [[894.673   0.    485.229]
 [  0.    896.159 633.036]
 [  0.      0.      1.   ]]


In [8]:
class CalibrationZeroSkew:
    """
    Adapted Zhang's camera calibration method for estimating intrinsic parameters
    from homographies between multiple planar templates in a single scene.
    This version assumes zero skew (s = 0).
    """
    
    def __init__(self):
        self.intrinsic_matrix = None
        self.homographies = []
    
    def _build_v_ij(self, H: np.ndarray, i: int, j: int) -> np.ndarray:
        """
        Compute the vector v_ij used in the Zhang-style calibration constraint:
        v_ij = [h1i*h1j, h1i*h2j + h2i*h1j, h2i*h2j, h3i*h1j + h1i*h3j, h3i*h2j + h2i*h3j, h3i*h3j]
        
        Args:
            H: homography matrix (3x3)
            i, j: column indices for the constraint
            
        Returns:
            v_ij: constraint vector (6,)
        """
        h = H.T  # transpose to access columns easily
        return np.array([
            h[i, 0] * h[j, 0],
            h[i, 0] * h[j, 1] + h[i, 1] * h[j, 0],
            h[i, 1] * h[j, 1],
            h[i, 0] * h[j, 2] + h[i, 2] * h[j, 0],
            h[i, 1] * h[j, 2] + h[i, 2] * h[j, 1],
            h[i, 2] * h[j, 2]
        ])
    
    def add_homography(self, H: np.ndarray):
        """
        Add a homography to the calibration dataset.
        
        Args:
            H: homography matrix (3x3)
        """
        if H.shape != (3, 3):
            raise ValueError("Homography must be a 3x3 matrix")
        self.homographies.append(H.copy())
    
    def add_homographies(self, Hs: list[np.ndarray]):
        """
        Add multiple homographies to the calibration dataset.
        
        Args:
            Hs: list of homography matrices (each 3x3)
        """
        for H in Hs:
            self.add_homography(H)
    
    def calibrate(self) -> np.ndarray:
        """
        Estimate the intrinsic matrix K from the stored homographies.
        
        Returns:
            K: intrinsic matrix (3x3)
            
        Raises:
            ValueError: if fewer than 3 homographies are available
        """
        if len(self.homographies) < 3:
            raise ValueError("At least 3 homographies are required for calibration")
        
        V = []

        # Add zero skew constraint: B12 = 0
        V.append([0, 1, 0, 0, 0, 0])

        for H in self.homographies:
            H_norm = H / H[2, 2]  # Normalize
            v12 = self._build_v_ij(H_norm, 0, 1)  # v_12
            v11 = self._build_v_ij(H_norm, 0, 0)  # v_11
            v22 = self._build_v_ij(H_norm, 1, 1)  # v_22
            V.append(v12)
            V.append(v11 - v22)

        V = np.vstack(V)

        print("V matrix for zero skew calibration:")
        print(V)
        
        # Solve Vb = 0 using SVD
        _, _, Vh = np.linalg.svd(V)
        b = Vh[-1, :]  # last row corresponds to smallest singular value
        
        print("b vector from SVD:")
        elements = ["B11", "B12", "B22", "B13", "B23", "B33"]
        for i in range(len(b)):
            print(f"{elements[i]}: {b[i]:.10f}")

        # b = [B11, B12, B22, B13, B23, B33]
        B11, B12, B22, B13, B23, B33 = b

        # Force B12 to be zero
        B12 = 0.0

        # Compute intrinsic parameters from B
        cy = (B12 * B13 - B11 * B23) / (B11 * B22 - B12**2)
        scale = B33 - (B13**2 + cy * (B12 * B13 - B11 * B23)) / B11
        fx = np.sqrt(scale / B11)
        fy = np.sqrt(scale * B11 / (B11 * B22 - B12**2))
        # s = -B12 * fx**2 * fy / scale
        s = 0  # Zero skew assumption
        cx = s * cy / fy - B13 * fx**2 / scale

        # Construct the intrinsic matrix
        K = np.array([
            [fx, s, cx],
            [0, fy, cy],
            [0, 0, 1]
        ])
        
        self.intrinsic_matrix = K
        return K
    
    def get_intrinsic_matrix(self) -> np.ndarray:
        """
        Get the estimated intrinsic matrix.
        
        Returns:
            K: intrinsic matrix (3x3) or None if calibration hasn't been performed
        """
        return self.intrinsic_matrix
    
    def reset(self):
        """
        Clear all stored homographies and reset the calibration state.
        """
        self.homographies.clear()
        self.intrinsic_matrix = None
    
    def get_num_homographies(self) -> int:
        """
        Get the number of stored homographies.
        
        Returns:
            Number of homographies
        """
        return len(self.homographies)

In [9]:
# Calibrate with zero skew assumption
calib_zero_skew = CalibrationZeroSkew()
calib_zero_skew.add_homographies(homographies_metric)
K_est_zero_skew = calib_zero_skew.calibrate()

# Print estimated intrinsic matrix
print("\nEstimated K from metric homographies with zero skew:\n", K_est_zero_skew)

# Print the correct intrinsic matrix for comparison
print("\nCorrect K from calibration file:\n", K)

V matrix for zero skew calibration:
[[ 0.     1.     0.     0.     0.     0.   ]
 [ 1.095  1.54  -1.045  0.001 -0.001 -0.   ]
 [ 0.931 -4.418 -2.219 -0.001 -0.002 -0.   ]
 [-0.527  1.687  0.714  0.     0.    -0.   ]
 [ 1.536  2.457 -1.85   0.    -0.001 -0.   ]
 [-0.818 -0.413  0.74  -0.    -0.     0.   ]
 [-0.441  3.112  0.387 -0.     0.    -0.   ]]
b vector from SVD:
B11: -0.0000009219
B12: 0.0000000041
B22: -0.0000009671
B13: 0.0003945146
B23: 0.0006073886
B33: -0.9999997377

Estimated K from metric homographies with zero skew:
 [[698.42    0.    427.931]
 [  0.    681.911 628.057]
 [  0.      0.      1.   ]]

Correct K from calibration file:
 [[894.673   0.    485.229]
 [  0.    896.159 633.036]
 [  0.      0.      1.   ]]


In [10]:
class CalibrationSimple:
    """
    Adapted Zhang's camera calibration method for estimating intrinsic parameters
    from homographies between multiple planar templates in a single scene.
    This version assumes zero skew (s = 0), known principal point (cx, cy), and
    square pixels (fx = fy).
    """
    
    def __init__(self):
        self.intrinsic_matrix = None
        self.homographies = []
    
    def _build_v_ij(self, H: np.ndarray, i: int, j: int) -> np.ndarray:
        """
        Compute the vector v_ij used in the Zhang-style calibration constraint:
        v_ij = [h1i*h1j, h1i*h2j + h2i*h1j, h2i*h2j, h3i*h1j + h1i*h3j, h3i*h2j + h2i*h3j, h3i*h3j]
        
        Args:
            H: homography matrix (3x3)
            i, j: column indices for the constraint
            
        Returns:
            v_ij: constraint vector (6,)
        """
        h = H.T  # transpose to access columns easily
        return np.array([
            h[i, 0] * h[j, 0],
            h[i, 0] * h[j, 1] + h[i, 1] * h[j, 0],
            h[i, 1] * h[j, 1],
            h[i, 0] * h[j, 2] + h[i, 2] * h[j, 0],
            h[i, 1] * h[j, 2] + h[i, 2] * h[j, 1],
            h[i, 2] * h[j, 2]
        ])
    
    def add_homography(self, H: np.ndarray):
        """
        Add a homography to the calibration dataset.
        
        Args:
            H: homography matrix (3x3)
        """
        if H.shape != (3, 3):
            raise ValueError("Homography must be a 3x3 matrix")
        self.homographies.append(H.copy())
    
    def add_homographies(self, Hs: list[np.ndarray]):
        """
        Add multiple homographies to the calibration dataset.
        
        Args:
            Hs: list of homography matrices (each 3x3)
        """
        for H in Hs:
            self.add_homography(H)
    
    def calibrate(self, principal_point=None) -> np.ndarray:
        """
        Estimate the intrinsic matrix K from the stored homographies.
        
        Args:
            principal_point: tuple (cx, cy) if known, otherwise assumes image center
            
        Returns:
            K: intrinsic matrix (3x3)
            
        Raises:
            ValueError: if fewer than 1 homography is available (since we have more constraints)
        """
        if len(self.homographies) < 1:  # Need fewer homographies now
            raise ValueError("At least 1 homography is required for calibration with known constraints")
        
        # Set principal point (default to image center if not provided)
        if principal_point is None:
            raise ValueError("Principal point (cx, cy) must be provided")
        else:
            cx, cy = principal_point
        
        V = []

        # Constraint 1: Zero skew (B12 = 0)
        V.append([0, 1, 0, 0, 0, 0])
        
        # Constraint 2: Square pixels (B11 = B22, so B11 - B22 = 0)
        V.append([1, 0, -1, 0, 0, 0])
        
        # Constraint 3: Known cx (B13 = -cx * B11)
        # This gives us: B13 + cx * B11 = 0
        V.append([cx, 0, 0, 1, 0, 0])
        
        # Constraint 4: Known cy (B23 = -cy * B22 = -cy * B11 since B11 = B22)
        # This gives us: B23 + cy * B11 = 0  
        V.append([cy, 0, 0, 0, 1, 0])

        # Add constraints from homographies (you might need fewer now)
        for H in self.homographies:
            H_norm = H / H[2, 2]  # Normalize
            v12 = self._build_v_ij(H_norm, 0, 1)  # v_12
            v11 = self._build_v_ij(H_norm, 0, 0)  # v_11  
            v22 = self._build_v_ij(H_norm, 1, 1)  # v_22
            
            V.append(v12)  # This is redundant now due to zero skew constraint
            V.append(v11 - v22)  # This is redundant due to square pixel constraint
            
        V = np.vstack(V)
        
        print("V matrix with all constraints:")
        print(V)
        
        # Solve Vb = 0 using SVD
        _, _, Vh = np.linalg.svd(V)
        b = Vh[-1, :]  # last row corresponds to smallest singular value
        
        print("b vector from SVD:")
        elements = ["B11", "B12", "B22", "B13", "B23", "B33"]
        for i in range(len(b)):
            print(f"{elements[i]}: {b[i]:.10f}")

        # b = [B11, B12, B22, B13, B23, B33]
        B11, B12, B22, B13, B23, B33 = b
        
        # Simplified parameter extraction
        # Since B11 = B22, B12 = 0, B13 = -cx*B11, B23 = -cy*B11
        
        # The focal length f (since fx = fy = f)
        scale = B33 + cx * B13 + cy * B23  # This should equal B33 - cx²*B11 - cy²*B11
        f = np.sqrt(scale / B11)
        
        # Construct the intrinsic matrix
        K = np.array([
            [f,  0, cx],
            [0,  f, cy], 
            [0,  0,  1]
        ])
        
        self.intrinsic_matrix = K
        return K
    
    def get_intrinsic_matrix(self) -> np.ndarray:
        """
        Get the estimated intrinsic matrix.
        
        Returns:
            K: intrinsic matrix (3x3) or None if calibration hasn't been performed
        """
        return self.intrinsic_matrix
    
    def reset(self):
        """
        Clear all stored homographies and reset the calibration state.
        """
        self.homographies.clear()
        self.intrinsic_matrix = None
    
    def get_num_homographies(self) -> int:
        """
        Get the number of stored homographies.
        
        Returns:
            Number of homographies
        """
        return len(self.homographies)
    
    def refine_intrinsics(self, keypoint_pairs: list, template_points: list, method='lm', verbose=False) -> np.ndarray:
        """
        Perform non-linear refinement of the intrinsic matrix using keypoint correspondences.
        
        Args:
            keypoint_pairs: List of keypoint correspondences for each template
                        Each element is an array of shape (N, 4) where each row is [x_scene, y_scene, x_template, y_template]
            template_points: List of 3D template coordinates for each template
                            Each element is an array of shape (N, 3) where each row is [X, Y, 0] (Z=0 for planar templates)
            method: Optimization method ('lm' for Levenberg-Marquardt, 'trf' for Trust Region)
            verbose: If True, print optimization progress
            
        Returns:
            K_refined: Refined intrinsic matrix (3x3)
        """
        if self.intrinsic_matrix is None:
            raise ValueError("Must perform initial calibration before refinement")
        
        if len(keypoint_pairs) != len(template_points) or len(keypoint_pairs) != len(self.homographies):
            raise ValueError("Number of keypoint pairs, template points, and homographies must match")
        
        # Initial parameters: [fx, fy, cx, cy] (no skew assumed)
        K_init = self.intrinsic_matrix
        fx_init, fy_init = K_init[0, 0], K_init[1, 1]
        cx_init, cy_init = K_init[0, 2], K_init[1, 2]
        
        # For simplified case, we can either optimize [f] (fx=fy) or [fx, fy]
        # Let's optimize just focal length f assuming fx = fy
        params_init = np.array([fx_init])
        
        def residual_function(params):
            f = params[0]
            
            # Construct current K matrix
            K_current = np.array([
                [f,  0, cx_init],
                [0,  f, cy_init], 
                [0,  0,  1]
            ])
            
            residuals = []
            
            # For each template
            for i, (kp_pairs, template_pts) in enumerate(zip(keypoint_pairs, template_points)):
                if len(kp_pairs) == 0:
                    continue
                    
                # Extract correspondences
                scene_points = kp_pairs[:, :2]  # [x_scene, y_scene]
                template_2d = kp_pairs[:, 2:]   # [x_template, y_template]
                
                # Convert template 2D to 3D (assuming Z=0 for planar templates)
                if template_pts.shape[1] == 2:
                    template_3d = np.hstack([template_pts, np.zeros((len(template_pts), 1))])
                else:
                    template_3d = template_pts
                
                # Estimate pose using current K
                # We need to find R,t such that scene_points = K @ (R @ template_3d.T + t)
                # For this, we can use PnP or extract from homography
                
                # Method 1: Extract pose from homography (assuming we have good homographies)
                H = self.homographies[i]
                
                # Decompose homography to get pose
                R, t = self._decompose_homography(H, K_current)
                
                # Project template points to image
                template_3d_h = np.hstack([template_3d, np.ones((len(template_3d), 1))])  # homogeneous
                
                # Transform to camera coordinates
                Rt = np.hstack([R, t.reshape(-1, 1)])
                P = K_current @ Rt
                
                # Project to image plane
                projected_h = P @ template_3d_h.T  # 3 x N
                projected_2d = projected_h[:2] / projected_h[2]  # 2 x N, normalize
                projected_2d = projected_2d.T  # N x 2
                
                # Compute reprojection errors
                errors = scene_points - projected_2d
                residuals.extend(errors.flatten())
            
            return np.array(residuals)
        
        # Perform optimization
        if verbose:
            print(f"Initial focal length: {fx_init:.3f}")
            print(f"Starting refinement with {len([r for kp in keypoint_pairs for r in kp])} correspondences...")
        
        result = least_squares(
            residual_function, 
            params_init,
            method=method,
            verbose=2 if verbose else 0,
            ftol=1e-12,
            xtol=1e-12,
            gtol=1e-12,
            max_nfev=1000
        )
        
        if verbose:
            print(f"Optimization converged: {result.success}")
            print(f"Final cost: {result.cost:.2e}")
            print(f"Function evaluations: {result.nfev}")
        
        # Extract refined parameters
        f_refined = result.x[0]
        
        # Construct refined intrinsic matrix
        K_refined = np.array([
            [f_refined, 0, cx_init],
            [0, f_refined, cy_init], 
            [0, 0, 1]
        ])
        
        if verbose:
            print(f"Refined focal length: {f_refined:.3f}")
            print(f"Change: {f_refined - fx_init:.3f}")
        
        self.intrinsic_matrix = K_refined
        return K_refined

    def _decompose_homography(self, H: np.ndarray, K: np.ndarray):
        """
        Decompose homography H = K @ [r1, r2, t] into rotation and translation.
        Assumes the template lies in the Z=0 plane.
        
        Args:
            H: Homography matrix (3x3)
            K: Intrinsic matrix (3x3)
            
        Returns:
            R: Rotation matrix (3x3)
            t: Translation vector (3,)
        """
        # Normalize homography
        H_norm = H / np.linalg.norm(np.linalg.inv(K) @ H[:, 0])
        
        # Extract rotation and translation
        K_inv = np.linalg.inv(K)
        h1, h2, h3 = H_norm[:, 0], H_norm[:, 1], H_norm[:, 2]
        
        r1 = K_inv @ h1
        r2 = K_inv @ h2
        t = K_inv @ h3
        
        # Ensure r1, r2 are unit vectors (approximately)
        r1 = r1 / np.linalg.norm(r1)
        r2 = r2 / np.linalg.norm(r2)
        
        # r3 = r1 x r2 to complete the rotation matrix
        r3 = np.cross(r1, r2)
        
        # Construct rotation matrix
        R = np.column_stack([r1, r2, r3])
        
        # Ensure R is a proper rotation matrix (det(R) = 1, orthogonal)
        U, _, Vt = np.linalg.svd(R)
        R = U @ Vt
        
        return R, t

In [11]:
# Use the simplified calibration method
calib_simple = CalibrationSimple()
calib_simple.add_homographies(homographies_metric)
K_est_simple = calib_simple.calibrate(principal_point=(485.2289588982219, 633.0361846891799))

# Print estimated intrinsic matrix
print("\nEstimated K from simplified calibration:\n", K_est_simple)

# Print the correct intrinsic matrix for comparison
print("\nCorrect K from calibration file:\n", K)

V matrix with all constraints:
[[  0.      1.      0.      0.      0.      0.   ]
 [  1.      0.     -1.      0.      0.      0.   ]
 [485.229   0.      0.      1.      0.      0.   ]
 [633.036   0.      0.      0.      1.      0.   ]
 [  1.095   1.54   -1.045   0.001  -0.001  -0.   ]
 [  0.931  -4.418  -2.219  -0.001  -0.002  -0.   ]
 [ -0.527   1.687   0.714   0.      0.     -0.   ]
 [  1.536   2.457  -1.85    0.     -0.001  -0.   ]
 [ -0.818  -0.413   0.74   -0.     -0.      0.   ]
 [ -0.441   3.112   0.387  -0.      0.     -0.   ]]
b vector from SVD:
B11: 0.0000007682
B12: -0.0000000011
B22: 0.0000007843
B13: -0.0003727292
B23: -0.0004862675
B33: 0.9999998123

Estimated K from simplified calibration:
 [[815.871   0.    485.229]
 [  0.    815.871 633.036]
 [  0.      0.      1.   ]]

Correct K from calibration file:
 [[894.673   0.    485.229]
 [  0.    896.159 633.036]
 [  0.      0.      1.   ]]


In [12]:
from src.matching import batch_template_match_for_calibration

# Example usage function
def calibrate_camera_from_templates(
    template_paths: list[str],
    scene_path: str,
    principal_point: tuple[float, float],
    template_real_sizes: list[tuple[float, float]] = None,
    extract_method: str = "SIFT",
    match_method: str = "BF",
    refine: bool = True,
    verbose: bool = True
) -> np.ndarray:
    """
    Complete pipeline: template matching -> initial calibration -> refinement.
    
    Args:
        template_paths: List of template image paths
        scene_path: Scene image path
        principal_point: Known principal point (cx, cy)
        template_real_sizes: Real-world template sizes
        extract_method: Feature extraction method
        match_method: Matching method
        refine: Whether to perform non-linear refinement
        verbose: Print detailed progress
    
    Returns:
        K: Refined camera intrinsic matrix
    """
    # Step 1: Batch template matching
    homographies, keypoint_pairs, template_points = batch_template_match_for_calibration(
        template_paths=template_paths,
        scene_path=scene_path,
        extract_method=extract_method,
        match_method=match_method,
        template_real_sizes=template_real_sizes,
        plot=False
    )
    
    if len(homographies) == 0:
        raise ValueError("No valid template matches found for calibration")
    
    # Step 2: Initial calibration
    calibrator = CalibrationSimple()
    calibrator.add_homographies(homographies)
    K_initial = calibrator.calibrate(principal_point=principal_point)
    
    if verbose:
        print("\nInitial calibration result:")
        print(K_initial)
    
    # Step 3: Non-linear refinement (if requested)
    if refine and len(keypoint_pairs) > 0:
        if verbose:
            print(f"\nPerforming non-linear refinement with {sum(len(kp) for kp in keypoint_pairs)} correspondences...")
        
        K_refined = calibrator.refine_intrinsics(
            keypoint_pairs=keypoint_pairs,
            template_points=template_points,
            verbose=verbose
        )
        
        if verbose:
            print("\nRefined calibration result:")
            print(K_refined)
        
        return K_refined
    else:
        return K_initial

In [13]:
# Test the complete pipeline with the scene
scene_id = "S12"
scene = data.get_scene(scene_id)
templates = [data.get_template(t_id) for t_id in data.get_all_scenes()[scene_id]]

template_paths = [os.path.join(project_root, t.path) for t in templates]
scene_path = os.path.join(project_root, scene.path)

# Known principal point from calibration file
principal_point = (485.2289588982219, 633.0361846891799)

# Perform the complete calibration pipeline
K_final = calibrate_camera_from_templates(
    template_paths=template_paths,
    scene_path=scene_path,
    principal_point=principal_point,
    template_real_sizes=[(t.height, t.width) for t in templates],
    extract_method='SIFT',
    match_method='BF',
    refine=True,
    verbose=True
)

# Print the final estimated intrinsic matrix
print("\nFinal estimated intrinsic matrix from complete pipeline:\n", K_final)

# Print the correct intrinsic matrix for comparison
print("\nCorrect K from calibration file:\n", K)

Processing 3 templates against scene...

Processing template 1/3: /home/stefano/projects/single-view-3d/assets/templates/template_02.jpg
✓ Template 1: 298 inliers, error: 0.69px

Processing template 2/3: /home/stefano/projects/single-view-3d/assets/templates/template_03.jpg
✓ Template 2: 143 inliers, error: 0.46px

Processing template 3/3: /home/stefano/projects/single-view-3d/assets/templates/template_04.jpg
✓ Template 3: 169 inliers, error: 0.37px

Successfully processed 3/3 templates
V matrix with all constraints:
[[  0.      1.      0.      0.      0.      0.   ]
 [  1.      0.     -1.      0.      0.      0.   ]
 [485.229   0.      0.      1.      0.      0.   ]
 [633.036   0.      0.      0.      1.      0.   ]
 [  0.126   0.177  -0.12    0.     -0.     -0.   ]
 [  0.106  -0.508  -0.256  -0.     -0.     -0.   ]
 [ -0.028   0.089   0.038   0.      0.     -0.   ]
 [  0.084   0.13   -0.095   0.     -0.     -0.   ]
 [ -0.041  -0.021   0.037  -0.     -0.      0.   ]
 [ -0.022   0.157 